In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.applications.vgg16 import VGG16

data_dir = "drive/MyDrive/new_violence"
im_size = 416
chunk_size = 32
# Define Model
def get_model():
  base_model = VGG16(input_shape = (im_size, im_size, 3), # Shape of our images
  include_top = False, # Leave out the last fully connected layer
  weights = 'imagenet')
  for layer in base_model.layers:
    layer.trainable = False
  x=keras.layers.Conv2D(64,3,activation='relu',padding='same')(base_model.output)
  x=keras.layers.Conv2D(32,3,activation='relu',padding='same')(x)
  x=keras.layers.Flatten()(x)
  x=keras.layers.Dense(64,activation='relu')(x)
  x=keras.layers.Dropout(0.5)(x)
  x=keras.layers.Dense(32,activation='relu')(x)
  x=keras.layers.Dense(16,activation='relu')(x)
  x=keras.layers.Dense(1,activation='sigmoid')(x)
  model=keras.models.Model(inputs=base_model.input,outputs=x)
  return model
# Load the dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(im_size, im_size),
    batch_size=chunk_size,
    validation_split=0.2,
    seed=123,
    subset="training"
)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  image_size=(im_size, im_size),
  batch_size=chunk_size,
  validation_split=0.2,
  seed=123,
  subset="validation"
)
# Normalize the data between 0 and 1
train_ds=train_ds.map(lambda x,y : (x/255.0,y))
valid_ds=valid_ds.map(lambda x,y : (x/255.0,y))

# model=keras.models.load_model("drive/MyDrive/violence_model")
# Define callbacks
my_callback = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
    keras.callbacks.ModelCheckpoint(filepath='drive/MyDrive/violence_model', monitor='val_accuracy', save_best_only=True)
]
model = get_model()
# Compile the model
adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss="binary_crossentropy", metrics=['accuracy'])

# Train the model
model.fit(train_ds,validation_data=valid_ds, epochs=50, callbacks=my_callback)


Found 11073 files belonging to 2 classes.
Using 8859 files for training.
Found 11073 files belonging to 2 classes.
Using 2214 files for validation.
Epoch 1/50
277/277 [==============================] - 2616s 9s/step - loss: 0.5012 - accuracy: 0.7432 - val_loss: 0.5022 - val_accuracy: 0.7823
Epoch 2/50
277/277 [==============================] - 173s 621ms/step - loss: 0.2742 - accuracy: 0.8954 - val_loss: 0.2155 - val_accuracy: 0.9097
Epoch 3/50
277/277 [==============================] - 181s 651ms/step - loss: 0.1906 - accuracy: 0.9309 - val_loss: 0.1697 - val_accuracy: 0.9304
Epoch 4/50
277/277 [==============================] - 172s 620ms/step - loss: 0.1376 - accuracy: 0.9513 - val_loss: 0.1647 - val_accuracy: 0.9327
Epoch 5/50
277/277 [==============================] - 172s 620ms/step - loss: 0.1083 - accuracy: 0.9631 - val_loss: 0.1592 - val_accuracy: 0.9422
Epoch 6/50
277/277 [==============================] - 173s 623ms/step - loss: 0.0829 - accuracy: 0.9732 - val_loss: 0.1345 -